In [0]:
# from cni_connectors import adls_gen1_connector as adls_conn
# from trs_control_field import trs_control_field as tcf

from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
import datetime
import json
import re

# var_adls_uri = adls_conn.adls_gen1_connect(spark, dbutils, scope="adls_gen1", dynamic_overwrite="dynamic")

var_adls_uri = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'

try:
  tables = json.loads(re.sub("\'", '\"', dbutils.widgets.get("tables")))
  dls = json.loads(re.sub("\'", '\"', dbutils.widgets.get("dls")))
  adf = json.loads(re.sub("\'", '\"', dbutils.widgets.get("adf")))
  
except:
  tables =  {
    "path_origin": "/usr/me/rais_vinculo",
    "path_destination":"/me/rais_vinculo"
  }
  
dls = {"folders":{"landing":"/tmp/dev/lnd","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","trusted":"/tmp/dev/trs"}}
# dls = {"folders":{"landing":"/lnd","error":"/err","staging":"/stg","log":"/log","raw":"/raw","trusted":"/trs"}}
adf = {
  'adf_factory_name': 'cnibigdatafactory', 
  'adf_pipeline_name': 'raw_trs_convenio_ensino_prof_carga_horaria',
  'adf_pipeline_run_id': 'p1',
  'adf_trigger_id': 't1',
  'adf_trigger_name': 'author_dev',
  'adf_trigger_time': '2020-10-13T15:00:06.0829994Z',
  'adf_trigger_type': 'Manual'
}



raw = dls['folders']['raw']
trs = dls['folders']['trusted']
var_adls_uri = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'

source = "{adl_path}{raw}{origin}".format(adl_path=var_adls_uri, raw=raw, origin=tables["path_origin"])
source

target = "{adl_path}{trs}{origin}".format(adl_path=var_adls_uri, trs=trs, origin=tables["path_destination"])
target

df_source = spark.read.parquet(source)


#-----------------
df_2008_to_2010 = df_source.filter((col("ANO") >= 2008) & (col("ANO") <= 2010))

# display(df_2008_to_2010)
# df_2008_to_2010.printSchema()


#------------------
# lit () nao e necessario

transformation_2008_to_2010 = {"CD_CBO94": when(lower(col("CD_CBO94")) == "ignorado", lit(-1)).otherwise(regexp_replace('CD_CBO94', 'CBO ', '')),
                               
                               # Para a coluna "CD_CBO94" when/quando o valor string "ignorado" for lower case, substitua por lit(-1), otherwise/caso contratio(regexp_replace( da coluna 'CD_CBO94' substituindo o valor 'CBO' por ' '))
                               # ANO = 2018, A coluna CD_CDO94 apresenta o valor 'IGNORADO'
                               
                               
                               "CD_SEXO": when(col('CD_SEXO') == 'MASCULINO', lit(1)).when(col('CD_SEXO') == 'FEMININO', lit(2)).otherwise(col('CD_SEXO')),
                               
                               # When/Quando a coluna 'CD_SEXO' == "MASCULINO", substitua o valor por 1 lit(1).When/Quando a coluna 'CD_SEXO' == "FEMININO", substitua o valor por 2 lit(2), otherwise/caso contrario substitua pela coluna col('CD_SEXO') 
                               
                               
                               "CD_CBO": when(lower(col("CD_CBO")) == "ignorado", lit("-1")).otherwise(regexp_replace('CD_CBO', 'CBO ', '')),
                               
                               # Para a coluna 'CD_CBO' em lower case com valores iguais a 'ignorado', substitua por -1 / lit(1), otherwise / caso contrario dar um regexp_replace(na coluna CD_CBO, substituindo 'CBO' por '') 
                                                              
                               "CD_CAUSA_AFASTAMENTO1": when(col('CD_CAUSA_AFASTAMENTO1') == '-1', lit('99')).otherwise(col('CD_CAUSA_AFASTAMENTO1')),
                               
                               "CD_CAUSA_AFASTAMENTO2": when(col('CD_CAUSA_AFASTAMENTO2') == '-1', lit('99')).otherwise(col('CD_CAUSA_AFASTAMENTO2')),
                               
                               "CD_CAUSA_AFASTAMENTO3": when(col('CD_CAUSA_AFASTAMENTO3') == '-1', lit('99')).otherwise(col('CD_CAUSA_AFASTAMENTO3')),
                               
                               "NR_DIA_INI_AF1": when(col('NR_DIA_INI_AF1') == '-1', lit('99')).otherwise(col('NR_DIA_INI_AF1')),
                               
                               "NR_DIA_INI_AF2": when(col('NR_DIA_INI_AF2') == '-1', lit('99')).otherwise(col('NR_DIA_INI_AF2')),
                               
                               "NR_DIA_INI_AF3": when(col('NR_DIA_INI_AF3') == '-1', lit('99')).otherwise(col('NR_DIA_INI_AF3')),
                               
                               "NR_MES_INI_AF1": when(lower(col('NR_MES_INI_AF1')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_INI_AF1')),
                               
                               # Para a coluna 'NR_MES_INI_AF1' transformando os valores em lower case, se os valores ('-1', 'ignorado') pertecerem a coluna 'NR_MES_INI_AF1', caso pertenca, substitua por 99 / lit(99), otherwise / caso contrario complete-o com os valores col('NR_MES_INI_AF1')
                               
                               
                               
                               
                               "NR_MES_INI_AF2": when(lower(col('NR_MES_INI_AF2')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_INI_AF2')),
                               
                               "NR_MES_INI_AF3": when(lower(col('NR_MES_INI_AF3')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_INI_AF3')),
                               
                               
                               
                               
                               "NR_DIA_FIM_AF1": when(col('NR_DIA_FIM_AF1') == -1, lit(99)).otherwise(col('NR_DIA_FIM_AF1')),
                               
                               "NR_DIA_FIM_AF2": when(col('NR_DIA_FIM_AF2') == -1, lit(99)).otherwise(col('NR_DIA_FIM_AF2')),
                               
                               "NR_DIA_FIM_AF3": when(col('NR_DIA_FIM_AF3') == -1, lit(99)).otherwise(col('NR_DIA_FIM_AF3')),
                               
                               "NR_MES_FIM_AF1": when(lower(col('NR_MES_FIM_AF1')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_FIM_AF1')),
                               
                               "NR_MES_FIM_AF2": when(lower(col('NR_MES_FIM_AF2')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_FIM_AF2')),
                               
                               "NR_MES_FIM_AF3": when(lower(col('NR_MES_FIM_AF3')).isin('-1','ignorado'), lit('99')).otherwise(col('NR_MES_FIM_AF3')),
                               
                               
                               "VL_IDADE": (months_between(lit(datetime.date(2009, 12, 31)).cast('date'), to_date(lpad('DT_DIA_MES_ANO_DATA_NASCIMENTO',8,'0'), 'ddMMyyyy')) / lit(12)).cast('int')
                               
                               # Com a funcao months_between foi definido a contagem em months desde a data 2009,12,31 em formato do type 'Date' a funcao lpad corta os numeros do lado esquerdo da coluna 'DT_DIA_MES_ANO_DATA_NASCIMENTO em formato ddMMyyy e depois / dividindo tudo por 12


In [0]:
for key in transformation_2008_to_2010:
  df_2008_to_2010 = df_2008_to_2010.withColumn(key, transformation_2008_to_2010[key])

In [0]:
columns_month = ["NR_MES_INI_AF1",
                 "NR_MES_INI_AF2",
                 "NR_MES_INI_AF3",
                 "NR_MES_FIM_AF1",
                 "NR_MES_FIM_AF2",
                 "NR_MES_FIM_AF3"]
 
for item in columns_month:
  df_2008_to_2010 = df_2008_to_2010\
  .withColumn(item, when(col("ANO") == 2010,
                        when(lower(col(item)) == "janeiro", lit('01'))\
                       .when(lower(col(item)) == "fevereiro", lit('02'))\
                       .when(lower(col(item)).isin("marco", "março"), lit('03'))\
                       .when(lower(col(item)) == "abril", lit('04'))\
                       .when(lower(col(item)) == "maio", lit('05'))\
                       .when(lower(col(item)) == "junho", lit('06'))\
                       .when(lower(col(item)) == "julho", lit('07'))\
                       .when(lower(col(item)) == "agosto", lit('08'))\
                       .when(lower(col(item)) == "setembro", lit('09'))\
                       .when(lower(col(item)) == "outubro", lit('10'))\
                       .when(lower(col(item)) == "novembro", lit('11'))\
                       .when(lower(col(item)) == "dezembro", lit('12'))\
                       .otherwise(col(item)))\
                   .otherwise(col(item)))
  
  
display(df_2008_to_2010.limit(3))  

nm_arq_in,CD_MUNICIPIO,CD_CNAE10_CLASSE,FL_VINCULO_ATIVO_3112,CD_TIPO_VINCULO,CD_MOTIVO_DESLIGAMENTO,CD_MES_DESLIGAMENTO,FL_IND_VINCULO_ALVARA,CD_TIPO_ADMISSAO,CD_TIPO_SALARIO,CD_CBO94,CD_GRAU_INSTRUCAO,CD_SEXO,CD_NACIONALIDADE,CD_RACA_COR,FL_IND_PORTADOR_DEFIC,CD_TAMANHO_ESTABELECIMENTO,CD_NATUREZA_JURIDICA,FL_IND_CEI_VINCULADO,CD_TIPO_ESTAB,FL_IND_ESTAB_PARTICIPA_PAT,FL_IND_SIMPLES,DT_DIA_MES_ANO_DATA_ADMISSAO,VL_REMUN_MEDIA_SM,VL_REMUN_MEDIA_NOM,VL_REMUN_DEZEMBRO_SM,VL_REMUN_DEZEMBRO_NOM,NR_MES_TEMPO_EMPREGO,QT_HORA_CONTRAT,VL_REMUN_ULTIMA_ANO_NOM,VL_REMUN_CONTRATUAL_NOM,ID_PIS,DT_DIA_MES_ANO_DATA_NASCIMENTO,ID_CTPS,ID_CPF,ID_CEI_VINCULADO,ID_CNPJ_CEI,ID_CNPJ_RAIZ,CD_TIPO_ESTAB_ID,ID_NOME_TRABALHADOR,DT_DIA_MES_ANO_DIA_DESLIGAMENTO,CD_CBO,CD_CNAE20_CLASSE,CD_CNAE20_SUBCLASSE,CD_TIPO_DEFIC,CD_CAUSA_AFASTAMENTO1,NR_DIA_INI_AF1,NR_MES_INI_AF1,NR_DIA_FIM_AF1,NR_MES_FIM_AF1,CD_CAUSA_AFASTAMENTO2,NR_DIA_INI_AF2,NR_MES_INI_AF2,NR_DIA_FIM_AF2,NR_MES_FIM_AF2,CD_CAUSA_AFASTAMENTO3,NR_DIA_INI_AF3,NR_MES_INI_AF3,NR_DIA_FIM_AF3,NR_MES_FIM_AF3,VL_DIAS_AFASTAMENTO,VL_IDADE,CD_IBGE_SUBSETOR,VL_ANO_CHEGADA_BRASIL,ID_CEPAO_ESTAB,CD_MUNICIPIO_TRAB,ID_RAZAO_SOCIAL,VL_REMUN_JANEIRO_NOM,VL_REMUN_FEVEREIRO_NOM,VL_REMUN_MARCO_NOM,VL_REMUN_ABRIL_NOM,VL_REMUN_MAIO_NOM,VL_REMUN_JUNHO_NOM,VL_REMUN_JULHO_NOM,VL_REMUN_AGOSTO_NOM,VL_REMUN_SETEMBRO_NOM,VL_REMUN_OUTUBRO_NOM,VL_REMUN_NOVEMBRO_NOM,FL_IND_TRAB_INTERMITENTE,FL_IND_TRAB_PARCIAL,FL_SINDICAL,VL_ANO_CHEGADA_BRASIL2,CD_CNAE20_DIVISAO,CD_UF,CD_CBO4,CD_BAIRROS_FORTALEZA,CD_BAIRROS_RJ,CD_BAIRROS_SP,CD_DISTRITOS_SP,CD_FAIXA_ETARIA,CD_FAIXA_HORA_CONTRAT,CD_FAIXA_REMUN_DEZEM_SM,CD_FAIXA_REMUN_MEDIA_SM,CD_FAIXA_TEMPO_EMPREGO,CD_REGIOES_ADM_DF,DT_MES_ADMISSAO,nr_reg,dh_insercao_raw,dh_arq_in,ANO
PB2009ID.TXT,250830,75116,1,30,00,00,2,0,1,CBO 55250,4,FEMININO,10,-1,1,8,1031,0,1,0,0,1121999,1.08,498.72,1.1,511.5,120.9,40,1.1,465.0,17036833627,10031957,16435,67516220400,0,8997611000168,8997611,1,MARIA DAS DORES LIMA DOS SANTOS,null,CBO 514215,84116,8411600,0,-1,null,-1,null,-1,-1,null,-1,null,-1,-1,null,-1,null,-1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1236950611498,2021-06-02T15:06:25.000+0000,2021-06-02T11:40:39.000+0000,2009
RS2009ID.TXT,430790,19399,0,10,21,03,2,2,1,CBO 80255,7,MASCULINO,10,2,1,4,2062,0,1,0,0,26012009,1.04,465.4,0.0,0.0,1.5,44,0.63,550.0,20365115961,18091989,8617762,1258022001,0,2318308000105,2318308,1,CAIRON MACIEL MARTINS,null,CBO 764305,15394,1539400,0,-1,null,-1,null,-1,-1,null,-1,null,-1,-1,null,-1,null,-1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,635655547437,2021-06-02T15:06:25.000+0000,2021-06-02T11:41:02.000+0000,2009
PB2009ID.TXT,250400,65234,1,10,00,00,2,0,1,CBO 39315,7,MASCULINO,10,2,1,5,2011,0,1,1,0,27091982,13.31,6149.14,13.5,6280.99,327.1,30,13.5,3344.0,10699129386,21011958,3614,13943669491,0,360305004100,360305,1,FRANCISCO DE ARAUJO MACEDO FILHO,null,CBO 413225,64239,6423900,0,40,21,1,23,1,-1,null,-1,null,-1,-1,null,-1,null,-1,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,377957126976,2021-06-02T15:06:25.000+0000,2021-06-02T11:40:39.000+0000,2009


In [0]:
df = df_2008_to_2010.union(df_2011_onwards.select(df_2008_to_2010.columns))

In [0]:
df = spark.createDataFrame([('1997-02-28 10:30:00', '1996-10-30')], ['date1', 'date2'])
df.select(months_between(df.date1, df.date2).alias('months')).show()

+----------+
 months|
+----------+
3.94959677|
+----------+

In [0]:
df3.write.partitionBy('ANO', 'CD_UF').save(path=target, format="parquet", mode='overwrite')
# Inside of Folder [Folder "ANO" -> Folder 'CD_UF_1',Folder 'CD_UF_2', Folder 'CD_UF_3' ]